In [1]:
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

In [4]:
from rpy2.robjects.packages import importr

utils = importr("utils")
utils.chooseCRANmirror(ind=1)
utils.install_packages('lme4')

%load_ext rpy2.ipython

R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/lme4_1.1-27.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 3311365 bytes (3.2 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to con

In [2]:
# Load data
conversations = pd.read_csv("results/intelligibility/conversations.csv")
conversations_melted = pd.read_csv("results/intelligibility/conversations_melted.csv")

# convert True/False to 0/1:
conversations.replace({False: 0, True: 1}, inplace=True)
conversations_melted.replace({False: 0, True: 1}, inplace=True)

conversations.head()


,utterance_id,speaker_code,tokens,pos,age,corpus,transcript_file,child_name,speaker_code_next,start_time_next,...,follow_up_start_time,follow_up_end_time,follow_up_is_speech_related,follow_up_is_intelligible,follow_up_speech_act,response_latency,response_latency_follow_up,has_response,response_is_clarification_request,pos_feedback
0,28,CHI,"[""let's"", 'get', 'some', 'glue', 'too', '.']","['v', 'pro:obj', 'v', 'qn', 'n', 'adv']",30,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/020324.cha,Bloom_Peter,MOT,96687.0,...,97154.0,100026.0,1,1,PF,0.0,467.0,1,0,1
1,50,CHI,"['i', 'get', 'some', 'glue', 'too', '.']","['pro:sub', 'v', 'qn', 'n', 'adv']",30,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/020324.cha,Bloom_Peter,MOT,148262.0,...,150114.0,151434.0,1,1,AA,0.0,1852.0,1,0,1
2,52,CHI,"['broken', '.']",['part'],30,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/020324.cha,Bloom_Peter,MOT,151434.0,...,155422.0,161449.0,1,1,ST,0.0,3988.0,1,0,1
3,58,CHI,"['huh', 'right', 'there', 'see', 'his', 'legs'...","['co', 'adv', 'adv', 'v', 'det:poss', 'n', 'v'...",30,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/020324.cha,Bloom_Peter,MOT,169064.0,...,172202.0,173521.0,1,1,AD,0.0,3138.0,1,0,1
4,78,CHI,"['hi', 'jenny', 'how', 'you', 'fine', 'hi', 'h...","['co', 'n:prop', 'pro:rel', 'pro:per', 'adv', ...",30,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/020324.cha,Bloom_Peter,MOT,243831.0,...,249125.0,249641.0,1,1,AA,0.0,5294.0,1,0,1


In [3]:
# Some children have very few data points:
counts = conversations.groupby("corpus").size()
counts

corpus
Bloom                   1042
Braunwald               1948
Brent                   2076
Forrester                205
Gleason                 6628
MPI-EVA-Manchester    139924
MacWhinney              9672
NewmanRatner             140
Peters                  5363
Providence             46594
Rollins                  113
Sachs                   2596
Snow                    6105
Soderstrom              1404
Thomas                111165
Tommerdahl              5287
Weist                   8395
dtype: int64

In [5]:
# normalize age
min_age, max_age = conversations["age"].min(), conversations["age"].max()
conversations["age"] = (conversations["age"] - min_age) / (max_age - min_age) * (1 - 0)

## Quality of communicative feedback/ Caregiver contingency


### Timing:

In [22]:
%%R -i conversations
library(lme4)

m_caregiver_contingency<-glmer('has_response ~ utt_is_intelligible * age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_caregiver_contingency))


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: has_response ~ utt_is_intelligible * age + (1 | child_name)
   Data: conversations

      AIC       BIC    logLik  deviance  df.resid 
 292958.1  293012.0 -146474.1  292948.1    348652 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-24.3846   0.0935   0.4958   0.5367   3.9572 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 3.596    1.896   
Number of obs: 348657, groups:  child_name, 52

Fixed effects:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)              2.33381    0.14693  15.884   <2e-16 ***
utt_is_intelligible      0.81883    0.04350  18.822   <2e-16 ***
age                      0.95097    0.07368  12.906   <2e-16 ***
utt_is_intelligible:age  0.05398    0.07930   0.681    0.496    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation o

### Clarification requests

In [13]:
conversations["response_is_no_clarification_request"] = conversations.response_is_clarification_request.replace({0:1, 1:0})
conversations

,utterance_id,speaker_code,tokens,pos,age,corpus,transcript_file,child_name,speaker_code_next,start_time_next,...,follow_up_end_time,follow_up_is_speech_related,follow_up_is_intelligible,follow_up_speech_act,response_latency,response_latency_follow_up,has_response,response_is_clarification_request,pos_feedback,response_is_no_clarification_request
0,28,CHI,"[""let's"", 'get', 'some', 'glue', 'too', '.']","['v', 'pro:obj', 'v', 'qn', 'n', 'adv']",0.526316,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/020324.cha,Bloom_Peter,MOT,96687.0,...,100026.0,1,1,PF,0.0,467.0,1,0,1,1
1,50,CHI,"['i', 'get', 'some', 'glue', 'too', '.']","['pro:sub', 'v', 'qn', 'n', 'adv']",0.526316,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/020324.cha,Bloom_Peter,MOT,148262.0,...,151434.0,1,1,AA,0.0,1852.0,1,0,1,1
2,52,CHI,"['broken', '.']",['part'],0.526316,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/020324.cha,Bloom_Peter,MOT,151434.0,...,161449.0,1,1,ST,0.0,3988.0,1,0,1,1
3,58,CHI,"['huh', 'right', 'there', 'see', 'his', 'legs'...","['co', 'adv', 'adv', 'v', 'det:poss', 'n', 'v'...",0.526316,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/020324.cha,Bloom_Peter,MOT,169064.0,...,173521.0,1,1,AD,0.0,3138.0,1,0,1,1
4,78,CHI,"['hi', 'jenny', 'how', 'you', 'fine', 'hi', 'h...","['co', 'n:prop', 'pro:rel', 'pro:per', 'adv', ...",0.526316,Bloom,/home/mitja/data/CHILDES/Bloom/Peter/020324.cha,Bloom_Peter,MOT,243831.0,...,249641.0,1,1,AA,0.0,5294.0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348652,602,CHI,"['go', 'in', '.']","['v', 'adv']",0.526316,Weist,/home/mitja/data/CHILDES/Weist/Matt/020504.cha,Weist_Matt,MOT,1845959.0,...,1850132.0,1,1,AA,0.0,3276.0,1,0,1,1
348653,604,CHI,"['yeah', '.']",['co'],0.526316,Weist,/home/mitja/data/CHILDES/Weist/Matt/020504.cha,Weist_Matt,MOT,1850132.0,...,1851495.0,1,1,AA,0.0,816.0,1,0,1,1
348654,606,CHI,"['yeah', '.']",['co'],0.526316,Weist,/home/mitja/data/CHILDES/Weist/Matt/020504.cha,Weist_Matt,MOT,1851495.0,...,1855070.0,1,1,SA,0.0,1047.0,1,0,1,1
348655,608,CHI,"['blankie', '.']",['n'],0.526316,Weist,/home/mitja/data/CHILDES/Weist/Matt/020504.cha,Weist_Matt,MOT,1855070.0,...,1866159.0,1,1,AA,0.0,9942.0,1,0,1,1


In [14]:
%%R -i conversations
library(lme4)

# TODO: filter out no response cases?
m_caregiver_contingency<-glmer('response_is_clarification_request ~ utt_is_intelligible * age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_caregiver_contingency))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: response_is_clarification_request ~ utt_is_intelligible * age +  
    (1 | child_name)
   Data: conversations

     AIC      BIC   logLik deviance df.resid 
 21180.4  21234.2 -10585.2  21170.4   348652 

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-0.359 -0.071 -0.070 -0.028 36.317 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 0.7341   0.8568  
Number of obs: 348657, groups:  child_name, 52

Fixed effects:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -3.62258    0.14789 -24.495   <2e-16 ***
utt_is_intelligible     -1.88158    0.10735 -17.528   <2e-16 ***
age                     -0.07421    0.17897  -0.415    0.678    
utt_is_intelligible:age -0.01593    0.18862  -0.084    0.933    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of F

### Combined (Positive Feedback = No pause, no clarification request)

In [24]:
%%R -i conversations
library(lme4)

m_caregiver_contingency<-glmer('pos_feedback ~ utt_is_intelligible * age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_caregiver_contingency))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: pos_feedback ~ utt_is_intelligible * age + (1 | child_name)
   Data: conversations

      AIC       BIC    logLik  deviance  df.resid 
 299553.7  299607.5 -149771.8  299543.7    348652 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-14.6641   0.1120   0.4955   0.5358   3.8471 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 2.137    1.462   
Number of obs: 348657, groups:  child_name, 52

Fixed effects:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)              1.69762    0.09499  17.871   <2e-16 ***
utt_is_intelligible      0.93039    0.03491  26.653   <2e-16 ***
age                      0.98015    0.06049  16.202   <2e-16 ***
utt_is_intelligible:age  0.01059    0.06182   0.171    0.864    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation o

## Effect of communicative feedback
### Positive Feedback: Timing

In [15]:
%%R -i conversations
library(lme4)

# conversations_child_intelligible = subset(conversations, is_intelligible==1)
# TODO: only intelligible ones?

m_child_contingency<-glmer('follow_up_is_intelligible ~ has_response * age + (1 | child_name)', data=conversations, family=binomial)
print(summary(m_child_contingency))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: follow_up_is_intelligible ~ has_response * age + (1 | child_name)
   Data: conversations

     AIC      BIC   logLik deviance df.resid 
194509.4 194563.2 -97249.7 194499.4   348652 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-11.2317   0.2031   0.2521   0.3220  10.8027 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 4.038    2.009   
Number of obs: 348657, groups:  child_name, 52

Fixed effects:
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)      -0.38933    0.08871  -4.389 1.14e-05 ***
has_response      0.58604    0.03479  16.843  < 2e-16 ***
age               2.47016    0.05671  43.559  < 2e-16 ***
has_response:age  0.26704    0.06185   4.318 1.58e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr) hs_

### Negative Feedback: Clarification requests

In [20]:
%%R -i conversations_melted
library(lme4)

conversations_melted_cr = subset(conversations_melted, response_is_clarification_request == 1)


m_child_contingency<-glmer('is_intelligible ~ is_follow_up * age + (1 | child_name) ', data=conversations_melted_cr, family=binomial)
print(summary(m_child_contingency))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: is_intelligible ~ is_follow_up * age + (1 | child_name)
   Data: conversations_melted_cr

     AIC      BIC   logLik deviance df.resid 
  4054.9   4086.0  -2022.4   4044.9     3735 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.3343 -0.9079  0.4006  0.6807  2.4031 

Random effects:
 Groups     Name        Variance Std.Dev.
 child_name (Intercept) 1.036    1.018   
Number of obs: 3740, groups:  child_name, 47

Fixed effects:
                  Estimate Std. Error z value Pr(>|z|)    
(Intercept)      -1.975988   0.327498  -6.034  1.6e-09 ***
is_follow_up      0.412259   0.364324   1.132    0.258    
age               0.080202   0.008657   9.264  < 2e-16 ***
is_follow_up:age  0.014781   0.012410   1.191    0.234    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr) is_fl_ age

In [17]:
%%R -i conversations_melted
library(lme4)

m_child_contingency<-glmer('is_intelligible ~ response_is_clarification_request * is_follow_up + (1 | age) + (1 | child_name) + (1 | is_follow_up)', data=conversations_melted, family=binomial)
print(summary(m_child_contingency))

R[write to console]: boundary (singular) fit: see ?isSingular



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: is_intelligible ~ response_is_clarification_request * is_follow_up +  
    (1 | age) + (1 | child_name) + (1 | is_follow_up)
   Data: conversations_melted

      AIC       BIC    logLik  deviance  df.resid 
 382930.9  383011.1 -191458.4  382916.9    697307 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-8.2847  0.2273  0.2477  0.3078 12.8115 

Random effects:
 Groups       Name        Variance Std.Dev.
 child_name   (Intercept) 3.478    1.865   
 age          (Intercept) 1.422    1.192   
 is_follow_up (Intercept) 0.000    0.000   
Number of obs: 697314, groups:  child_name, 52; age, 8; is_follow_up, 2

Fixed effects:
                                               Estimate Std. Error z value
(Intercept)                                     1.27267    0.05492  23.175
response_is_clarification_request              -1.90881    0.03540 -53.927
is